In [18]:
%pip install pandas numpy tqdm statsmodels

  Obtaining dependency information for statsmodels from https://files.pythonhosted.org/packages/d5/24/cb8b04e66d15cd99a1ed138a8061a4a9a11f227a65df0c6ea8a02a02f106/statsmodels-0.14.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for scipy!=1.9.2,>=1.8 from https://files.pythonhosted.org/packages/8e/ee/8a26858ca517e9c64f84b4c7734b89bda8e63bec85c3d2f432d225bb1886/scipy-1.14.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.1 MB/s eta 0:00:00
  Obtaining dependency information for patsy>=0.5.6 from https://files.pythonhosted.org/packages/43/f3/1d311a09c34f14f5973bb0bb0dc3a6e007e1eda90b5492d082689936ca51/patsy-0.5.6-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 4.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 kB 3.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [30]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from statsmodels.discrete.discrete_model import Logit

In [12]:
df = pd.read_csv('./Data/23-24-problem_logs.csv')

In [13]:
df.sort_values(by=['user_xid','skill_id','start_time'])

,problem_log_id,user_xid,old_problem_id,skill_id,start_time,discrete_score,attempt_count,hint_count,saw_answer
0,190360611,3812471,1785247,211,2024-05-16 16:49:07.425 -0400,0,5,0,False
1,190382132,4086708,372227,149,2024-05-17 11:34:32.246 -0400,1,1,0,False
2,185619619,3947429,289580,181,2024-02-21 13:36:24.096 -0500,1,1,0,False
3,184911130,3850768,909447,2282,2024-02-09 14:11:06.805 -0500,1,1,0,False
4,190321521,2926553,1989847,84,2024-05-16 09:05:57.234 -0400,1,1,0,False
...,...,...,...,...,...,...,...,...,...
12075729,176271644,3327391,1395263,136,2023-09-06 10:31:48.295 -0400,1,1,0,False
12075730,176271725,3327391,1395279,136,2023-09-06 10:32:15.151 -0400,1,1,0,False
12075731,176271573,3327391,1395248,136,2023-09-06 10:31:08.934 -0400,1,1,0,False
12075732,178169637,453384,1459780,164,2023-10-04 11:10:36.366 -0400,1,1,0,False


In [16]:
df['skill_id_cat'] = df['skill_id'].astype('category', copy=False)

0            211
1            149
2            181
3           2282
4             84
            ... 
12075729     136
12075730     136
12075731     136
12075732     164
12075733     164
Name: skill_id, Length: 12075734, dtype: category
Categories (525, int64): [1, 2, 3, 4, ..., 2282, 2283, 2284, 2285]

In [23]:
res = []
for name, group in tqdm(df.groupby(by=['user_xid', 'skill_id'])):
    wins = 0
    fails = 0
    for idx, row in group.iterrows():
        
        res.append([wins,fails,row['skill_id'],row['discrete_score']])
        
        if row['discrete_score'] == 1:
            wins += 1
        else:
            fails += 1
             
    

100%|██████████| 1424943/1424943 [10:27<00:00, 2270.77it/s]


TypeError: data type 'category' not understood

In [24]:
res_df = pd.DataFrame(res, columns=['w','f','skill_id','correct'])

In [26]:
res_df['skill_id'] = res_df['skill_id'].astype('category')

In [27]:
res_df

,w,f,skill_id,correct
0,0,0,237,1
1,1,0,237,1
2,2,0,237,0
3,0,0,1126,1
4,1,0,1126,0
...,...,...,...,...
12075729,5,0,372,1
12075730,6,0,372,1
12075731,7,0,372,1
12075732,8,0,372,1


In [38]:
mod = Logit.from_formula('correct ~ w*C(skill_id) + f*C(skill_id)',data=res_df.head(n=250000))

In [40]:
r = mod.fit()

/home/mplee/Documents/python-envs/TESTING/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:2385: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/home/mplee/Documents/python-envs/TESTING/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:2443: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q * linpred)))


         Current function value: inf
         Iterations: 35


LinAlgError: Singular matrix